# Eye movement panel of the poster

Looks only at right eye cameras

# Get data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import matplotlib as mpl
import seaborn as sns
from pathlib import Path
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.measure import EllipseModel
from mpl_toolkits.axes_grid1 import make_axes_locatable

import flexiznam as flz
from v1_depth_analysis.config import PROJECT
import v1_depth_analysis as vda
from cottage_analysis.eye_tracking import analysis as analeyesis
from cottage_analysis.eye_tracking import eye_model_fitting as emf

MOTION_CUTOFF = 1  # in degrees

## Get data for all sessions

In [ ]:
# get session list
raw_path = Path(flz.PARAMETERS["data_root"]["raw"])
processed_path = Path(flz.PARAMETERS["data_root"]["processed"])
flm_sess = flz.get_flexilims_session(project_id=PROJECT)

recordings = vda.get_recordings(protocol="SpheresPermTubeReward", flm_sess=flm_sess)
datasets = vda.get_datasets(
    recordings, dataset_type="camera", dataset_name_contains="_eye", flm_sess=flm_sess
)
# keep only right eye cameras
datasets = [ds for ds in datasets if 'right' in ds.dataset_name]
camera_datasets = {ds.full_name: ds for ds in datasets}


In [ ]:
# get the data
data_by_recording = dict()
sampling_by_recording = dict()
no_behaviour = []
for cam_name, camera in camera_datasets.items():
    dlc_res, ellipse = analeyesis.get_data(
        camera,
        flexilims_session=flm_sess,
        likelihood_threshold=0.88,
        rsquare_threshold=0.99,
        error_threshold=3,
    )
    try:
        data, sampling = analeyesis.add_behaviour(
            camera, dlc_res, ellipse, speed_threshold=0.01, log_speeds=False, verbose=False,
        )
    except FileNotFoundError as err:
        warnings.warn(f'No data for {cam_name}')
        no_behaviour.append(cam_name)
    assert "valid" in data.columns
    # add trial number
    depth = np.array(data.depth.values, copy=True)
    depth[np.isnan(depth)] = -9999
    depth = np.round(depth, 2)
    trials_border = np.diff(np.hstack([-9999, depth]))
    trials_onset = np.where(trials_border > 5000)[0]
    trials_offset = np.where(trials_border < -5000)[0]
    trial_id = np.zeros(depth.shape) + np.nan
    for itrial, (on, off) in enumerate(zip(trials_onset, trials_offset)):
        trial_id[on:off] = itrial
    data['trial_id'] = trial_id
    data_by_recording[cam_name] = data
    sampling_by_recording[cam_name] =sampling
print(f'Loaded {len(data_by_recording)- len(no_behaviour)}/{len(datasets)} recordings ')


In [ ]:
# get the eye tracking fit
eye_parameters_by_recording = dict()
eye_rotations_by_recording = dict()
no_tracking = []
for cam_name, camera in camera_datasets.items():
    camera_save_folder = processed_path / camera.path / camera.dataset_name
    try:
        eye_parameters_by_recording[cam_name] = np.load(camera_save_folder / f"{camera.dataset_name}_eye_parameters.npz")
    except FileNotFoundError as err:
        warnings.warn(f'No data for {cam_name}')
        no_tracking.append(cam_name)
        continue
    eye_rotations_by_recording[cam_name] = np.load(
        camera_save_folder / f"{camera.dataset_name}_eye_rotation_by_frame.npy"
    )
print(f'Loaded {len(eye_rotations_by_recording)}/{len(datasets)} recordings ')

In [ ]:
# Get camera extrinsics
calibration_folder = processed_path / PROJECT / "Calibrations"
calib_data = dict()
for cam_name in ["RightEyeCam", "LeftEyeCam"]:
    calib_data[cam_name.lower()] = dict()
    folder = calibration_folder / cam_name
    folder = list(folder.glob("*xtrinsics_flat"))[0]  # case is inconsistent
    folder = folder / "20220818" / "aruco5_5mm"
    assert folder.exists()
    for trial in folder.glob("trial*"):
        fname = str(trial / "camera_extrinsics_flat.yml")
        s = cv2.FileStorage()
        s.open(fname, cv2.FileStorage_READ)
        rvec = s.getNode("rvec").mat()
        tvec = s.getNode("tvec").mat()
        calib_data[cam_name.lower()][trial.name] = dict(rvec=rvec, tvec=tvec)
# take median across trials
extrinsics = dict()
for cam, trials in calib_data.items():
    extrinsics[cam] = dict()
    for w in ["rvec", "tvec"]:
        extrinsics[cam][w] = np.median(
            np.vstack([d[w].flatten() for d in trials.values()]), axis=0
        )

In [ ]:
# Get azimuth elevation
azel_by_recording = dict()
bad_number = []
valid_data = dict()
for cam_name, eye_rotation in eye_rotations_by_recording.items():
    if cam_name not in data_by_recording:
        print(f'Skipping bad session {cam_name}')
    camera = camera_datasets[cam_name]
    # get the camera we need for this acq and build tform matrix
    extrin = extrinsics[camera.dataset_name.replace("_", "")[:-3]]
    rmat, jac = cv2.Rodrigues(extrin["rvec"])
    gaze_vec = np.vstack([emf.get_gaze_vector(p[0], p[1]) for p in eye_rotation])
    world_gaze = emf.convert_to_world(gaze_vec, rmat=rmat)
    azimuth, elevation = emf.gaze_to_azel(world_gaze)

    # add that to dataframe
    # I need to cut the last few because of extra SI triggers
    
    azel_by_recording = np.vstack([azimuth, elevation])
    n = len(data_by_recording[cam_name])
    difference = len(azimuth) - n
    if (difference < 0) or (difference > 5):
        bad_number.append([cam_name, difference])
    else:
        valid_data[cam_name] = data_by_recording[cam_name].copy()
        valid_data[cam_name]['azimuth'] = np.rad2deg(azimuth[:n] - np.nanmedian(azimuth[:n]))
        valid_data[cam_name]['elevation'] = np.rad2deg(elevation[:n] - np.nanmedian(elevation[:n]))
print(f'Got {len(valid_data)} recordings at the end')
bad_number

In [ ]:
# add some useful measure
for cam_name, data in valid_data.items():
    data['delta_az'] = data.azimuth.diff()
    data['delta_el'] = data.elevation.diff()
    data['nasal'] = np.nan
    data['temporal'] = np.nan
    data.loc[data.delta_az > 0, 'nasal'] = np.abs(data.loc[data.delta_az > MOTION_CUTOFF, 'delta_az'])
    data.loc[data.delta_az < 0, 'temporal'] = np.abs(data.loc[data.delta_az < -MOTION_CUTOFF, 'delta_az'])
    data['angle_of_motion'] = np.rad2deg(np.arctan2(np.deg2rad(data.delta_el),
                                         np.deg2rad(data.delta_az)))
    data['amplitude_of_motion'] = np.linalg.norm(np.vstack([data.delta_az, data.delta_el]), axis=0)
    data['angle_when_moving'] = np.nan
    data['saccade'] = data['amplitude_of_motion'] > (80/sampling)
    moving = data.amplitude_of_motion > MOTION_CUTOFF
    data.loc[moving, 'angle_when_moving'] = data.loc[moving, 'amplitude_of_motion']

In [ ]:
# remove session where the fit failed
badly_fitted =  "PZAH6.4b_S20220519_R183410_SpheresPermTubeReward_right_eye_camera"
_ = valid_data.pop(badly_fitted)


In [ ]:
# aggregate data by session

by_session = []
av_trials = False
for cam_name, data in valid_data.items():
    in_corridor = data[~np.isnan(data.depth)].copy()
    by_trial = in_corridor.groupby(["trial_id"])
    m_by_trial = by_trial.aggregate(np.nanmean)
    if av_trials:
        # now average trial averages
        m_by_depth = m_by_trial.groupby(['depth']).aggregate(np.nanmean)
    else:
        m_by_depth = m_by_trial
    m_by_depth = m_by_depth.reset_index()
    m_by_depth['session'] = cam_name
    by_session.append(m_by_depth)

by_session = pd.concat(by_session, ignore_index=True)
by_session.shape

# add cumulative motion
cumul_session = []
for cam_name, data in valid_data.items():
    in_corridor = data[~np.isnan(data.depth)].copy()
    by_trial = in_corridor.groupby(["trial_id"])
    m_by_trial = by_trial.aggregate(np.nansum)
    if av_trials:
        # now average trial averages
        m_by_depth = m_by_trial.groupby(['depth']).aggregate(np.nanmean)
    else:
        m_by_depth = m_by_trial
    m_by_depth = m_by_depth.reset_index()
    m_by_depth['session'] = cam_name
    cumul_session.append(m_by_depth)
by_session_cumul = pd.concat(cumul_session, ignore_index=True)


## Get example session

In [ ]:
# camera_full_name = "PZAH6.4b_S20220419_R145152_SpheresPermTubeReward_right_eye_camera"
start_frame = 12000

camera_full_name = "PZAH6.4b_S20220512_R190248_SpheresPermTubeReward_right_eye_camera"

camera = [ds for ds in datasets if ds.full_name == camera_full_name]
camera = camera[0]
print(f"Analysing {' from '.join(camera.genealogy[::-1])}")
data = valid_data[camera_full_name]
eye_parameters = eye_parameters_by_recording[camera_full_name]
eye_rotations = eye_rotations_by_recording[camera_full_name]

In [ ]:
# Get example frame
video_file = camera.path_full / camera.extra_attributes["video_file"]
dlc_ds_name = "_".join(
    list(camera.genealogy[:-1]) + ["dlc_tracking", camera.dataset_name, "data", "0"]
)
dlc_ds = flz.Dataset.from_flexilims(name=dlc_ds_name, flexilims_session=flm_sess)
cropping = dlc_ds.extra_attributes["cropping"]
cam_data = cv2.VideoCapture(str(video_file))
cam_data.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
ret, frame = cam_data.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cam_data.release()
gray = gray[cropping[2] : cropping[3], cropping[0] : cropping[1]]

# Plot the panel

In [ ]:
## Example eye

## binned data 
elli = pd.DataFrame(data[data.valid], copy=True)
count, bin_edges_x, bin_edges_y = np.histogram2d(
    elli.pupil_x, elli.pupil_y, bins=(25, 25)
)
elli["bin_id_x"] = bin_edges_x.searchsorted(elli.pupil_x.values)
elli["bin_id_y"] = bin_edges_y.searchsorted(elli.pupil_y.values)
binned_ellipses = elli.groupby(["bin_id_x", "bin_id_y"])
ns = binned_ellipses.valid.aggregate(len)
binned_ellipses = binned_ellipses.aggregate(np.nanmedian)
mat = (np.zeros((26,26, 2))+ np.nan)
for i_pos, (pos, series) in enumerate(binned_ellipses.iterrows()):
    mat[pos[1], pos[0]] = series[['azimuth', 'elevation']].values

# get other things we want to plot
eye_centre=eye_parameters['eye_centre']
f_z0=eye_parameters['f_z0']
ellipse_model = emf.reproj_ellipse(*eye_rotations[start_frame], eye_centre=eye_centre, f_z0=f_z0)
ref = data.iloc[start_frame][['reflection_x', 'reflection_y']].values


In [ ]:
# get depth color
depth_list = np.unique(data.depth)
depth_list = depth_list[~np.isnan(depth_list)]
cmap = mpl.cm.cool.reversed()
line_colors = []
norm = mpl.colors.Normalize(vmin=np.log(min(depth_list)), vmax=np.log(max(depth_list)))
col_dict = dict()
for depth in depth_list:
    rgba_color = cmap(norm(np.log(depth)), bytes=True)
    rgba_color = tuple(it / 255 for it in rgba_color)
    line_colors.append(rgba_color)
    col_dict[depth] = rgba_color


In [ ]:
def plot_example_eye(ax, fig):
    labels = ["azimuth", "elevation"]
    v = binned_ellipses[["pupil_x", "pupil_y"]].values
    lims = np.vstack([np.nanmin(v, axis=0), np.nanmax(v, axis=0)]) + ref
    circ_coord = ellipse_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(1, 2)
    vmin, vmax = np.quantile(gray, [0.01, 0.8])
    ax.imshow(gray, cmap="gray", vmin=vmin, vmax=vmax, zorder=-1)
    img = ax.imshow(
        mat[..., 0],
        extent=np.hstack([lims[:, 0], lims[::-1, 1]]),
        cmap="RdBu_r",
        vmin=-20,
        vmax=20,
        zorder=10,
    )
    ax.plot(
        circ_coord[:, 0],
        circ_coord[:, 1],
        label="Reprojection",
        color="lightblue",
        alpha=0.5,
        zorder=5,
    )
    pupil_c = np.array(ellipse_model.params[:2])
    ax.plot(*(eye_centre + ref), marker="o", ms=5, mfc="k", mec="none", zorder=1)
    ax.plot(*(pupil_c + ref), marker="o", ms=5, mfc="none", color="lightblue", alpha=0.5)
    ax.plot(
        *[(np.array([eye_centre[i], pupil_c[i]]) + ref[i]) for i in range(2)],
        color="lightblue",
        zorder=2,
        alpha=0.5
    )
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cb = fig.colorbar(img, cax=cax)
    cb.set_label("Azimuth (degrees)")
    ax.set_xlim([gray.shape[1], 0])
    ax.set_ylim([gray.shape[0] - 80, 50])
    ax.text(x=0.5, y=0.8, s="Dorsal", color="white", transform=ax.transAxes)
    ax.text(x=0.6, y=0.1, s="Nasal", color="white", transform=ax.transAxes)
    ax.axis("off")



In [ ]:
# time course 
colors = [np.array([141,160,203])/255, np.array([252,141,98])/255]
def plot_time_course(ax, add_ampl=False, n_samples=int(120*sampling), begin=20000, colors=colors):
    
    ax.axhline(0, color='grey', alpha=0.5, lw=1)
    time = np.arange(n_samples) / sampling
    
    azim = data.azimuth
    elev = data.elevation
    ax.plot(
        time,elev[begin : begin + n_samples], color=colors[1], label='Elevation'
    )
    ax.plot(
        time, azim[begin : begin + n_samples], color=colors[0], label='Azimuth'
    )
    if add_ampl:
        ax.plot(
        time, data.amplitude_of_motion[begin : begin + n_samples], color='k', label='Amplitude'
    )

    ax.set_xlabel("Time (s)")
    ax.set_ylabel(r"$\Delta$angle (degrees)")
    ax.set_xlim(0, time.max())
    ax.set_ylim(-20, 20)
    ax.set_xticks(np.arange(0, time.max(), 60))
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.legend(loc='upper right')
    divider = make_axes_locatable(ax)
    ax = divider.append_axes("right", size="10%", pad=0.05)
    bins = np.arange(-25, 25)
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.hist(
        elev,
        orientation="horizontal",
        density=True,
        bins=bins,
        color=colors[1],
        histtype="step",
        lw=2,
    )
    ax.hist(
        azim,
        orientation="horizontal",
        density=True,
        bins=bins,
        color=colors[0],
        histtype="step",
        lw=2,
    )
    ax.set_xlabel("Density")
    ax.set_yticks([])


In [ ]:
from matplotlib.collections import PolyCollection
from matplotlib.legend_handler import HandlerTuple

def label_violins(ax, title=None, hue_order = ['nasal', 'temporal']):
    idepth = 0
    handles = []
    for ind, violin in enumerate(ax.findobj(PolyCollection)):
        color = line_colors[idepth]
        if ind % 2 != 0:
            color = 0.5 + 0.5 * np.array(color)  # make whiter
            idepth+=1
        violin.set_facecolor(color)
        handles.append(plt.Rectangle((0, 0), 0, 0, facecolor=color, edgecolor='black'))
    ax.legend(handles=[tuple(handles[::2]), tuple(handles[1::2])], labels=hue_order,
            title=title, handlelength=4, handler_map={tuple: HandlerTuple(ndivide=None, pad=0)})


In [ ]:
fig = plt.figure(figsize=(17.7165, 7.87402))
# Eye tracking video example
ax = plt.subplot2grid((5, 12), (0, 0), rowspan=2, colspan=2)
plot_example_eye(ax, fig)

ax_timecourse = plt.subplot2grid((5, 12), (0, 3), rowspan=2, colspan=4)
plot_time_course(ax_timecourse, add_ampl=False)
ax_timecourse.set_ylim([-15, 15])
# plot of position by depth
ax_medposition = plt.subplot2grid((5, 12), (0, 7), rowspan=2, colspan=2)

in_corridor = data[~np.isnan(data.depth)]
by_trial = in_corridor.groupby(["trial_id"])
m_by_trial = by_trial.aggregate(np.nanmean)
by_depth = m_by_trial.reset_index().groupby("depth")

m_by_depth = by_depth.aggregate(np.nanmean)
std_by_depth = by_depth.aggregate(np.nanstd)
for idepth, depth in enumerate(depth_list):
    ax_medposition.errorbar(
        m_by_depth.loc[depth].azimuth,
        m_by_depth.loc[depth].elevation,
        xerr=std_by_depth.loc[depth].azimuth,
        yerr=std_by_depth.loc[depth].elevation,
        marker="o",
        color=line_colors[idepth],
        label=int(depth)
    )
ax_medposition.legend(loc="upper right", ncol=2)
ax_medposition.set_xlabel(r'Azimuth ($\circ$)')
ax_medposition.set_xlim([-6, 6])
ax_medposition.set_ylim([-2, 4])
ax_medposition.set_ylabel(r'Elevation ($\circ$)')


ax_pos = [(0, 9), (0, 10), (0, 11), (1, 9), (1, 10)]
width = 20
bins = np.arange(-180, 180, width)

for idepth, depth in enumerate(depth_list):
    ax_polar = plt.subplot2grid((5, 12), ax_pos[idepth], rowspan=1, colspan=1, projection='polar')
    d = data.loc[data.depth==depth]
    h, _ = np.histogram(d[d.amplitude_of_motion > (80/sampling)].angle_of_motion, bins=bins)
    ax_polar.bar(np.deg2rad(bins[:-1]), h, width=np.deg2rad(width), color=line_colors[idepth])
    ax_polar.set_rlim(0, 14)
    ax_polar.set_xticks(np.arange(0, np.pi, np.pi/2))
    ax_polar.set_xticklabels(['Nasal', 'Dorsal'])
    ax_polar.set_yticks([14])
    ax_polar.grid(False)

ax_all_sess = plt.subplot2grid((5, 6), (2, 0), rowspan=3, colspan=3)
azel_df = pd.DataFrame(dict(Axis=['Azimuth'] * len(by_session) + ['Elevation'] * len(by_session),
                         Position = np.hstack([by_session.azimuth.values, by_session.elevation.values]),
                         Depth=np.hstack([by_session.depth, by_session.depth]).astype(int)))
ax_all_sess.axhline(0, color='gray', alpha=0.5, zorder=-10)
sns.violinplot(azel_df, x='Depth', y='Position', hue='Axis', ax=ax_all_sess, split=True, hue_order=['Azimuth', 'Elevation'])
label_violins(ax_all_sess, title=None, hue_order=['Azimuth', 'Elevation'])
ax_all_sess.set_xlabel('Depth')
ax_all_sess.set_ylabel(r'Average position ($\circ$)')


ax_all_sess = plt.subplot2grid((5, 6), (2, 3), rowspan=3, colspan=3)
motion_df = pd.DataFrame(dict(direction=['Nasal'] * len(by_session_cumul) + ['Temporal'] * len(by_session_cumul),
                         motion = np.hstack([by_session_cumul.saccade.values, by_session_cumul.saccade.values]) ,
                         depth=np.hstack([by_session.depth, by_session.depth]).astype(int)))

sns.violinplot(motion_df, x='depth', y='motion', hue='direction', ax=ax_all_sess,  hue_order=['Nasal', 'Temporal'], split=True)
ax_all_sess.set_xlabel('Depth')
ax_all_sess.set_ylabel(r'Cumulative motion per trial ($\circ$)')
label_violins(ax_all_sess, title='Direction',  hue_order=['Nasal', 'Temporal'])



fig.subplots_adjust(wspace=0.8, hspace=1)

In [ ]:
valid_data.shape()

In [ ]:
ax_pos = [(0, 9), (0, 10), (0, 11), (1, 9), (1, 10)]
width = 20
bins = np.arange(-180, 180, width)
fig = plt.figure(figsize=(10,5))

for imouse, mouse in enumerate(['PZAH', 'PZAG']):
    for idepth, depth in enumerate(depth_list):
        ax_polar = fig.add_subplot(2, 5, idepth+1 + imouse*5, projection='polar')
        if idepth == 0:
            ax_polar.set_ylabel(mouse)
        h = np.zeros(len(bins)-1)
        for c, dat in valid_data.items():
            if not c.startswith(mouse):
                continue
            d = dat.loc[dat.depth==depth]
            hd, _ = np.histogram(d[d.amplitude_of_motion > (80/sampling)].angle_of_motion, bins=bins)
            h += hd
        ax_polar.bar(np.deg2rad(bins[:-1]), h, width=np.deg2rad(width), color=line_colors[idepth])
        # ax_polar.set_rlim(0, 14)
        ax_polar.set_xticks(np.arange(0, np.pi, np.pi/2))
        ax_polar.set_xticklabels(['Nasal', 'Dorsal'])
        #ax_polar.set_yticks([14])
        ax_polar.grid(False)


In [ ]:
plt.hist(data.amplitude_of_motion, bins=np.arange(0,30,0.5))
plt.semilogy()

In [ ]:
for cm, d in valid_data.items():
    plt.figure(figsize=(2, 2))
    ax_polar = plt.subplot(1,1,1, projection='polar')
    print(cm)
    plt.title(cm)
    width = 20
    bins = np.arange(-180, 180, width)
    h, _ = np.histogram(d.angle_of_motion[d.amplitude_of_motion > 5], bins=bins)
    ax_polar.bar(np.deg2rad(bins[:-1]), h, width=np.deg2rad(width))


In [ ]:
plt.close('all')

In [ ]:
ax = plt.subplot(1,1,1, projection="polar")
width=10
binned, be = np.histogram(data.angle_of_motion[data.amplitude_of_motion > 1], bins=np.arange(-180, 180, width))
allbinned = np.zeros(binned.shape)
for data in valid_data.values():
    binned, be = np.histogram(data.angle_of_motion[data.amplitude_of_motion > 1], bins=np.arange(-180, 180, width), weights=data.amplitude_of_motion[data.amplitude_of_motion > 1])
    allbinned += binned
ax.bar(np.deg2rad(be[:-1]), allbinned, width=np.deg2rad(width), bottom=0)

In [ ]:
by_trial.nasal